# ICBC Project Warm-Up: Feature Engineering

In [1]:
import os
import numpy as  np
import pandas as pd

In [31]:
wd = '/Users/ewenwang/OneDrive/IBM/Project_ICBC/code_wu'
os.chdir(wd)
datafile = 'lc_2016Q3_clean.csv'
data = pd.read_csv(datafile, header=0, encoding='latin-1')
data.info()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95210 entries, 0 to 95209
Data columns (total 90 columns):
loan_amnt                         95210 non-null int64
term                              95210 non-null object
int_rate                          95210 non-null float64
installment                       95210 non-null float64
emp_title                         89516 non-null object
emp_length                        95210 non-null int64
home_ownership                    95210 non-null object
annual_inc                        95210 non-null float64
verification_status               95210 non-null object
issue_d                           95210 non-null object
loan_status                       95210 non-null int64
pymnt_plan                        95210 non-null object
desc                              6 non-null object
purpose                           95210 non-null object
title                             90065 non-null object
zip_code                          95210 non-null object

In [32]:
data.head(4)

,loan_amnt,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,15000,36 months,13.99,512.60,Fiscal Director,2,RENT,55000.0,Not Verified,01/09/2016,...,0,5,100.0,33.3,0,0,147587,140492,30200,108587
1,2600,36 months,8.99,82.67,Loaner Coordinator,3,RENT,35000.0,Source Verified,01/09/2016,...,0,0,100.0,0.0,0,0,73227,19497,50200,20527
2,32200,60 months,21.49,880.02,warehouse/supervisor,10,MORTGAGE,65000.0,Not Verified,01/09/2016,...,0,2,100.0,40.0,1,0,199969,24934,13900,15886
3,10000,36 months,11.49,329.72,Teacher,10,OWN,55900.0,Not Verified,01/09/2016,...,0,6,100.0,66.7,0,0,68831,38085,13500,36431


In [33]:
target = data['loan_status']

In [34]:
data = data.drop(['loan_status'], 1)

In [41]:
data.select_dtypes(include=['object']).iloc[:,0:20].head(4)

,home_ownership,verification_status,issue_d,desc,purpose,title,addr_state,earliest_cr_line,application_type
0,RENT,Not Verified,01/09/2016,NaN,debt_consolidation,Debt consolidation,TN,01/01/1996,INDIVIDUAL
1,RENT,Source Verified,01/09/2016,NaN,moving,Moving and relocation,OR,01/09/2009,INDIVIDUAL
2,MORTGAGE,Not Verified,01/09/2016,NaN,credit_card,Credit card refinancing,CA,01/11/1992,INDIVIDUAL
3,OWN,Not Verified,01/09/2016,NaN,debt_consolidation,Debt consolidation,TX,01/07/1987,INDIVIDUAL


In [36]:
data.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95210 entries, 0 to 95209
Data columns (total 15 columns):
term                         95210 non-null object
emp_title                    89516 non-null object
home_ownership               95210 non-null object
verification_status          95210 non-null object
issue_d                      95210 non-null object
pymnt_plan                   95210 non-null object
desc                         6 non-null object
purpose                      95210 non-null object
title                        90065 non-null object
zip_code                     95210 non-null object
addr_state                   95210 non-null object
earliest_cr_line             95210 non-null object
initial_list_status          95210 non-null object
application_type             95210 non-null object
verification_status_joint    501 non-null object
dtypes: object(15)
memory usage: 10.9+ MB


In [37]:
## strings
# titles
senior = ['Manager', 'Director', 'Senior', 'manager', 'Supervisor', 'Lead', 'Sr.', 'Officer', 'Sr',
          'supervisor', 'Administrator', 'Management', 'Executive', 'VP', 'Vice', 'President', 'Chief',
          'director', 'Admin', 'Administrative', 'Director,' 'MANAGER', 'lead', 'officer', 'Leader',
          'Manager,', 'Mgr', 'Head', 'associate', 'Associate', 'leader', 'Partner', 'Manger',
          'SR']
middle = ['Coordinator', 'Operations', 'Consultant', 'operator', 'Operator', 'consultant',
          'Representative', 'coordinator', 'Advisor',  'Counselor', 'Instructor', 'District', 'Architect',
          'Planner', 'Technologist', 'Master', 'Therapist', 'therapist', 'Professor', 'Investigator',
          'Coach']
junior = ['Specialist', 'Analyst', 'Assistant', 'Sales', 'Engineer', 'Technician', 'Support', 'specialist',
          'Account', 'service', 'technician', 'Clerk', 'Nurse', 'assistant', 'Maintenance', 'driver', 'Driver',
          'clerk', 'Client', 'Staff', 'Worker', 'HR', 'Teacher', 'Designer', 'nurse', 'worker', 'Accountant',
          'Inspector', 'agent', 'teacher', 'Member', 'Trainer', 'Secretary', 'Auditor', 'Sergeant', 'Processor',
          'customer', 'SPECIALIST', 'Banker', 'Student', 'ASSISTANT']

def replace_titles(title):
    """ Replacing all titles with 3, 2, 1, 0."""
    x = str(title['emp_title'])
    x = x.split()
    if any(i in senior for i in x):
        return 3
    elif any(i in middle for i in x):
        return 2
    elif any(i in junior for i in x):
        return 1
    else:
        return 0

def makeString(dataframe):
    """ Convert strings to numbers."""
    dataframe['term'] = dataframe['term'].map(
        {' 36 months': 0, ' 60 months': 1})
    dataframe['zip_code'] = dataframe['zip_code'].str.strip('xx').astype(int)
    dataframe['emp_title'] = pd.DataFrame(
        dataframe['emp_title']).apply(replace_titles, axis=1)
    dataframe['pymnt_plan'] = dataframe['pymnt_plan'].map({'n': 0, 'y': 1})
    dataframe['initial_list_status'] = dataframe[
        'initial_list_status'].map({'f': 0, 'w': 1})
    dataframe['verification_status_joint'] = dataframe[
        'verification_status_joint'].map({'nan': 0, 'Not Verified': 1})
    return None


## categories
cates = ['home_ownership', 'verification_status',
         'purpose', 'addr_state', 'title', 'application_type']

def makeDummy(dataframe):
    """ Convert categories to dummies. """
    features = pd.DataFrame()
    for cate in cates:
        features = pd.get_dummies(dataframe[cate])
        dataframe = dataframe.join(features)
        dataframe = dataframe.drop([cate], 1)
    dataframe[DateTime] = dataframe[DateTime].apply(date2delta, axis=0)
    return None

# datetime
DateTime = ['issue_d']

def date2delta(time):
    date = pd.to_datetime(time)
    delta = date - date.min()
    return delta.dt.days

In [39]:
makeString(data)
makeDummy(data)

In [44]:
    data['term'] = data['term'].map(
        {' 36 months': 0, ' 60 months': 1})
    data['emp_title'] = pd.DataFrame(
        data['emp_title']).apply(replace_titles, axis=1)
    data['pymnt_plan'] = data['pymnt_plan'].map({'n': 0, 'y': 1})
    data['initial_list_status'] = data[
        'initial_list_status'].map({'f': 0, 'w': 1})
    data['verification_status_joint'] = data[
        'verification_status_joint'].map({'nan': 0, 'Not Verified': 1})

In [48]:
#data['zip_code'] = data['zip_code'].strip('xx').astype(int)
makeDummy(data)

In [49]:
data.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95210 entries, 0 to 95209
Data columns (total 9 columns):
home_ownership         95210 non-null object
verification_status    95210 non-null object
issue_d                95210 non-null object
desc                   6 non-null object
purpose                95210 non-null object
title                  90065 non-null object
addr_state             95210 non-null object
earliest_cr_line       95210 non-null object
application_type       95210 non-null object
dtypes: object(9)
memory usage: 6.5+ MB


### Strings

In [9]:
data.term.unique()
#data.iloc[:,3].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [17]:
data['term'] = data['term'].map({' 36 months': 0, ' 60 months': 1})

In [11]:
#data['int_rate'] = data['int_rate'].str.strip('%').astype(float)

In [12]:
#data['revol_util'] = data['revol_util'].str.strip('%').astype(float)

In [13]:
data['emp_length'].unique()

array(['2 years', '3 years', '10+ years', '5 years', nan, '4 years',
       '7 years', '8 years', '9 years', '1 year', '6 years', '< 1 year'], dtype=object)

In [14]:
empl = data['emp_length']

In [15]:
empl.unique()

array(['2 years', '3 years', '10+ years', '5 years', nan, '4 years',
       '7 years', '8 years', '9 years', '1 year', '6 years', '< 1 year'], dtype=object)

In [16]:
empl = empl.str.extract('(\d+)').astype(float)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [17]:
data['emp_length'] = data['emp_length'].str.extract('(\d+)').astype(float)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [18]:
#data['zip_code'].unique()

In [19]:
data['zip_code'] = data['zip_code'].str.strip('xx').astype(int)

In [20]:
v = data['emp_title'].value_counts()

v.head()

Teacher             1851
Manager             1626
Owner                957
Supervisor           756
Registered Nurse     727
Name: emp_title, dtype: int64

In [80]:
v.nlargest(60).tail()

Administrator         121
Teacher               121
Professor             120
Account Executive     120
Executive Director    116
Name: emp_title, dtype: int64

In [87]:
titles = v.index.astype(list)

In [91]:
splite_titles = [words for segments in titles for words in segments.split()]

In [99]:
len(splite_titles)

90858

In [98]:
title_dict = dict((i, splite_titles.count(i)) for i in splite_titles)

In [105]:
import operator

sorted_title = sorted(title_dict.items(), key=operator.itemgetter(1), reverse=True)

In [106]:
sorted_title

[('Manager', 2787),
 ('Director', 1459),
 ('Specialist', 1345),
 ('Senior', 1276),
 ('of', 1214),
 ('manager', 965),
 ('Analyst', 927),
 ('Assistant', 900),
 ('Sales', 785),
 ('Coordinator', 778),
 ('Supervisor', 710),
 ('Engineer', 669),
 ('Lead', 662),
 ('Technician', 616),
 ('Service', 608),
 ('and', 608),
 ('Sr.', 574),
 ('Operations', 543),
 ('Associate', 515),
 ('Officer', 494),
 ('Tech', 484),
 ('Services', 469),
 ('Support', 464),
 ('Business', 461),
 ('&', 404),
 ('Sr', 403),
 ('tech', 398),
 ('II', 391),
 ('Consultant', 389),
 ('specialist', 373),
 ('IT', 364),
 ('supervisor', 328),
 ('operator', 319),
 ('Project', 306),
 ('Customer', 304),
 ('Administrator', 297),
 ('Operator', 294),
 ('Management', 287),
 ('Account', 284),
 ('Executive', 282),
 ('service', 278),
 ('technician', 275),
 ('Program', 269),
 ('Clerk', 266),
 ('Systems', 262),
 ('Nurse', 259),
 ('Development', 248),
 ('Office', 247),
 ('VP', 247),
 ('Technical', 244),
 ('Clinical', 243),
 ('Vice', 241),
 ('Medica

In [107]:
sorted_title.index

<function list.index>

In [23]:
title_data = pd.DataFrame(data['emp_title'])
title_data.head()

,emp_title
0,Fiscal Director
1,Loaner Coordinator
2,warehouse/supervisor
3,Teacher
4,SERVICE MGR


In [142]:
type(title_data)

pandas.core.frame.DataFrame

In [21]:
senior = ['Manager', 'Director', 'Senior', 'manager', 'Supervisor', 'Lead', 'Sr.', 'Officer', 'Sr',
         'supervisor', 'Administrator','Management', 'Executive', 'VP', 'Vice', 'President', 'Chief',
         'director', 'Admin', 'Administrative', 'Director,' 'MANAGER', 'lead', 'officer','Leader',
         'Manager,', 'Mgr', 'Head', 'associate', 'Associate', 'leader', 'Partner', 'Manger', 
         'SR']
middle = ['Coordinator', 'Operations', 'Consultant', 'operator', 'Operator', 'consultant', 
         'Representative', 'coordinator', 'Advisor',  'Counselor', 'Instructor', 'District', 'Architect', 
          'Planner', 'Technologist', 'Master', 'Therapist', 'therapist', 'Professor', 'Investigator', 
         'Coach']
junior = ['Specialist', 'Analyst', 'Assistant', 'Sales', 'Engineer', 'Technician', 'Support', 'specialist', 
          'Account', 'service', 'technician', 'Clerk', 'Nurse','assistant', 'Maintenance', 'driver', 'Driver', 
          'clerk', 'Client', 'Staff', 'Worker', 'HR', 'Teacher', 'Designer', 'nurse', 'worker', 'Accountant', 
          'Inspector', 'agent', 'teacher', 'Member', 'Trainer', 'Secretary', 'Auditor', 'Sergeant', 'Processor', 
          'customer', 'SPECIALIST', 'Banker', 'Student', 'ASSISTANT']

#replacing all titles with 3, 2, 1, 0
def replace_titles(title):
    x = str(title['emp_title'])
    x = x.split()
    if any(i in senior for i in x):
        return 3
    elif any(i in middle for i in x):
        return 2
    elif any(i in junior for i in x):
        return 1
    else:
        return 0

In [24]:
title_data = title_data.apply(replace_titles, axis = 1)
title_data.head()

0    3
1    2
2    0
3    1
4    0
dtype: int64

In [25]:
data['emp_title'] = title_data

### Categories

In [20]:
data['grade'].unique()

array(['C', 'B', 'D', 'E', 'A', 'F', 'G'], dtype=object)

In [21]:
data['grade'] = data['grade'].map({
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7})

In [22]:
data['sub_grade'].unique()

array(['C3', 'B1', 'D5', 'B5', 'C2', 'C4', 'D4', 'E4', 'B4', 'C1', 'B2',
       'A1', 'D3', 'C5', 'A3', 'A5', 'B3', 'E1', 'A4', 'F2', 'D1', 'D2',
       'A2', 'G3', 'F4', 'G1', 'E2', 'E5', 'F1', 'E3', 'F5', 'G4', 'F3',
       'G2', 'G5'], dtype=object)

In [23]:
data['sub_grade'] = data['sub_grade'].map({
    'A1': 10,
    'A2': 12,
    'A3': 14,
    'A4': 16,
    'A5': 18,
    'B1': 20,
    'B2': 22,
    'B3': 24,
    'B4': 26,
    'B5': 28,
    'C1': 30,
    'C2': 32,
    'C3': 34,
    'C4': 36,
    'C5': 38,
    'D1': 40,
    'D2': 42,
    'D3': 44,
    'D4': 46,
    'D5': 48,
    'E1': 50,
    'E2': 52,
    'E3': 54,
    'E4': 56,
    'E5': 58,
    'F1': 60,
    'F2': 62,
    'F3': 64,
    'F4': 66,
    'F5': 68,
    'G1': 70,
    'G2': 72,
    'G3': 74,
    'G4': 76,
    'G5': 78,
    })

In [27]:
data['home_ownership'].unique()

array(['RENT', 'MORTGAGE', 'OWN', 'ANY'], dtype=object)

In [28]:
#home = data['home_ownership']
features = pd.get_dummies(home)
#data = data.drop(['home_ownership'], 1)

NameError: name 'home' is not defined

In [177]:
features.head()

,ANY,MORTGAGE,OWN,RENT
0,0,0,0,1
1,0,0,0,1
2,0,1,0,0
3,0,0,1,0
4,0,0,0,1


In [176]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95210 entries, 0 to 95209
Data columns (total 4 columns):
ANY         95210 non-null uint8
MORTGAGE    95210 non-null uint8
OWN         95210 non-null uint8
RENT        95210 non-null uint8
dtypes: uint8(4)
memory usage: 372.0 KB


In [29]:
data['verification_status'].unique()

array(['Not Verified', 'Source Verified', 'Verified'], dtype=object)

In [30]:
add = pd.get_dummies(data['verification_status'])
features = pd.concat([features, add])
data = data.drop(['verification_status'], 1)

NameError: name 'features' is not defined

In [29]:
data['pymnt_plan'].unique()

array(['n', 'y'], dtype=object)

In [30]:
data['pymnt_plan'] = data['pymnt_plan'].map({'n': 0, 'y': 1})

In [31]:
data['purpose'].unique()

array(['debt_consolidation', 'moving', 'credit_card', 'medical', 'house',
       'major_purchase', 'home_improvement', 'other', 'car',
       'small_business', 'vacation', 'renewable_energy', 'wedding'], dtype=object)

In [32]:
add2 = pd.get_dummies(data['purpose'])
features = pd.concat([features, add2])
data = data.drop(['purpose'], 1)

In [33]:
data['addr_state'].unique()

array(['TN', 'OR', 'CA', 'TX', 'VA', 'IL', 'FL', 'OH', 'IN', 'WA', 'NJ',
       'NC', 'LA', 'NY', 'SC', 'CT', 'GA', 'MI', 'NH', 'OK', 'ME', 'RI',
       'AL', 'PA', 'MA', 'NV', 'CO', 'AZ', 'DC', 'AR', 'WI', 'MT', 'ID',
       'NE', 'HI', 'MD', 'MN', 'KS', 'MO', 'NM', 'VT', 'KY', 'UT', 'SD',
       'ND', 'MS', 'AK', 'WY', 'DE', 'WV'], dtype=object)

In [34]:
add3 = pd.get_dummies(data['addr_state'])
features = pd.concat([features, add3])
data = data.drop(['addr_state'], 1)

In [35]:
data['initial_list_status'].unique()

array(['f', 'w'], dtype=object)

In [36]:
data['initial_list_status'] = data['initial_list_status'].map({'f': 0, 'w': 1})

In [37]:
data['title'].unique()

array(['Debt consolidation', 'Moving and relocation',
       'Credit card refinancing', 'Medical expenses', 'Home buying',
       'Major purchase', 'Home improvement', 'Other', 'Car financing',
       'Business', 'Vacation', nan, 'Green loan'], dtype=object)

In [38]:
add4 = pd.get_dummies(data['title'])
features = pd.concat([features, add4])
data = data.drop(['title'], 1)

In [39]:
data['application_type'].unique()

array(['INDIVIDUAL', 'JOINT', 'DIRECT_PAY'], dtype=object)

In [40]:
add5 = pd.get_dummies(data['application_type'])
features = pd.concat([features, add5])
data = data.drop(['application_type'], 1)

In [41]:
data['verification_status_joint'].unique()

array([nan, 'Not Verified'], dtype=object)

In [42]:
data['verification_status_joint'] = data['verification_status_joint'].map({'nan': 0, 'Not Verified': 1})

### Datatime

In [44]:
type(data['issue_d'])

pandas.core.series.Series

In [47]:
#issue = pd.DataFrame(data['issue_d'])
issue = pd.to_datetime(data['issue_d'])

In [49]:
type(issue)
issue.head()

0   2017-09-16
1   2017-09-16
2   2017-09-16
3   2017-09-16
4   2017-09-16
Name: issue_d, dtype: datetime64[ns]

In [52]:
delta = issue - issue.min()
delta.head()

0   62 days
1   62 days
2   62 days
3   62 days
4   62 days
Name: issue_d, dtype: timedelta64[ns]

In [57]:
delta.dt.days.unique()

array([62, 31,  0])

In [58]:
data['issue_d'] = delta.dt.days

In [59]:
data['last_pymnt_d'].unique()

array(['6/17/2017', '1/17/2017', '4/17/2017', '3/17/2017', '11/16/2017',
       '5/17/2017', '10/16/2017', '2/17/2017', '12/16/2017', '9/16/2017',
       '8/16/2017', '7/16/2017'], dtype=object)

In [60]:
def date2delta(datetime):
    date = pd.to_datetime(datetime)
    delta = date - date.min()
    return delta.dt.days

In [61]:
lastp = date2delta(data['last_pymnt_d'])
lastp.unique()

array([151,   0,  90,  59, 303, 120, 272,  31, 333, 242, 211, 180])

In [62]:
data['last_pymnt_d'] = date2delta(data['last_pymnt_d'])

In [63]:
data['last_credit_pull_d'].unique()

array(['6/17/2017', '1/17/2017', '5/17/2017', '4/17/2017', '2/17/2017',
       '10/16/2017', '11/16/2017', '9/16/2017', '3/17/2017', '12/16/2017',
       nan, '8/16/2017', '7/16/2017', '6/16/2017'], dtype=object)

In [64]:
data['last_credit_pull_d'] = date2delta(data['last_credit_pull_d'])

In [65]:
data['last_credit_pull_d'].unique()

array([ 151.,    0.,  120.,   90.,   31.,  272.,  303.,  242.,   59.,
        333.,   nan,  211.,  180.,  150.])

In [66]:
data['earliest_cr_line'].unique()[1:10]

array(['9/9/2017', '11/1/1992', '7/1/1987', '7/1/1996', '8/4/2017',
       '2/4/2017', '6/1/2017', '6/1/1993', '8/1/1986'], dtype=object)

In [68]:
data['earliest_cr_line'] = date2delta(data['earliest_cr_line'])
data['earliest_cr_line'].unique()[1:10]

array([27798, 18720, 16770, 20058, 27762, 27581, 27698, 18932, 16436])

In [70]:
data['next_pymnt_d'].unique()

array(['7/17/2017', nan, '6/17/2017', '8/17/2017'], dtype=object)

In [71]:
data['next_pymnt_d'] = date2delta(data['next_pymnt_d'])
data['next_pymnt_d'].unique()

array([ 30.,  nan,   0.,  61.])

### Combine all features

In [167]:
data.info

<bound method DataFrame.info of        loan_amnt  funded_amnt  funded_amnt_inv  term  int_rate  installment  \
0          15000        15000            15000     0      0.14       512.60   
1           2600         2600             2600     0      0.09        82.67   
2          32200        32200            32200     1      0.21       880.02   
3          10000        10000            10000     0      0.11       329.72   
4           6000         6000             6000     0      0.13       203.59   
5          30000        30000            30000     1      0.14       697.90   
6           3250         3250             3250     0      0.13       110.28   
7           3000         3000             3000     0      0.14       103.25   
8          35000        35000            35000     0      0.20      1300.55   
9          20000        20000            20000     0      0.14       688.33   
10         20150        20150            20150     1      0.25       591.32   
11         12000    

In [170]:
dataset = pd.concat([data, features])
dataset = pd.concat([target, dataset])
dataset.info

<bound method DataFrame.info of          0  AK  AL  ANY  AR  AZ  Business  CA  CO  CT    ...     total_pymnt  \
0      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
1      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
2      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
3      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
4      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
5      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
6      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
7      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
8      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
9      0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
10     0.0 NaN NaN  NaN NaN NaN       NaN NaN NaN NaN    ...             NaN   
11     0